In [1]:
import requests

In [2]:
import json

In [18]:
from main import flatten, fix_values
import os

In [6]:
proj_endpoint = 'https://api.optimizely.com/v2/projects'
exp_endpoint = 'https://api.optimizely.com/v2/experiments'

In [7]:
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

In [4]:
def read_endpoint(endpoint, headers_set, params_set=None):
    try:
        response = requests.get(endpoint, headers=headers_set, params=params_set)
        response_text = json.loads(response.text)
        response.raise_for_status()

    except requests.exceptions.HTTPError as err:
        print(err)

    return response_text

In [5]:
def generate_projects(project_endpoint, project_headers):
    # get all projects
    j_proj = read_endpoint(endpoint=project_endpoint, headers_set=project_headers)

    return j_proj

In [27]:
str(datetime.now())

'2019-02-08 13:42:05.606595'

In [26]:
from datetime import datetime

In [32]:
def generate_experiments(project_id, experiment_endpoint, experiment_headers, experiment_params):
    # get all experiments from one project
    j_exp = read_endpoint(endpoint=experiment_endpoint, headers_set=experiment_headers, params_set=experiment_params)

    # loop for all experiments in this project
    experiment_id_list = []
    upload_exp_list = []
    for exp in j_exp:
        exp['project_id'] = project_id
        exp['upload_ts'] = str(datetime.now())
        upload_exp_list.append(flatten(exp, {}, ''))
        experiment_id_list.append(exp['id'])
        
    return upload_exp_list, experiment_id_list

In [11]:
project_id_list = []

In [12]:
for p in generate_projects(proj_endpoint, headers):
    project_id_list.append(p['id'])

In [13]:
project_id_list

[9965963792, 10347872424, 10427612860, 10429921250, 10849384554]

In [14]:
params = (
    ('project_id', 9965963792),
    ('per_page', 100),
) 

In [33]:
exp_list, exp_id_list = generate_experiments(9965963792, exp_endpoint, headers, params)

In [24]:
for exp in exp_list:
    if exp['created'] >= '2018-10-23T17:48:46.952180Z':
        print(exp['id'])

11938601024
11966791385
11992110033


In [28]:
type(exp_list[0]['created'])

str

In [30]:
exp_list[0]['created'] <= str(datetime.now())

True

In [34]:
exp_list

[{'allocation_policy': 'manual',
  'audience_conditions': '["and", {"audience_id": 10769441047}]',
  'campaign_id': 10755791168,
  'created': '2018-05-23T17:48:35.952180Z',
  'description': 'Testing alternate experience for FT sign up form',
  'earliest': '2018-05-25T17:32:25.742000Z',
  'holdback': 0,
  'id': 10756790433,
  'is_classic': False,
  'last_modified': '2018-06-03T04:59:26.187040Z',
  'latest': '2018-06-03T04:59:28.088920Z',
  'metrics_aggregator': 'unique',
  'metrics_event_id': 10769480069,
  'metrics_scope': 'visitor',
  'metrics_winning_direction': 'increasing',
  'name': 'FT Signup Variations',
  'project_id': 9965963792,
  'status': 'paused',
  'type': 'a/b',
  'upload_ts': '2019-02-08 13:43:51.990231',
  'url_targeting_activation_type': 'url_changed',
  'url_targeting_conditions': '["and", ["or", {"match_type": "simple", "type": "url", "value": "freetrial.infusionsoft.com/setup"}, {"match_type": "simple", "type": "url", "value": "freetrial.infusionsoft.com/signup"}]]

In [43]:
import popelines

In [45]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

In [46]:
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)


# test specific experiment

In [64]:
result_endpoint = 'https://api.optimizely.com/v2/experiments/10796678057/timeseries'
# result_endpoint = 'https://api.optimizely.com/v2/experiments/10756790433/timeseries'
response_ts = requests.get(result_endpoint, headers=headers)

In [67]:
j_ts = json.loads(response_ts.text)
new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')

flattened_metrics = []

for metric in new_j_ts['metrics']:
    for result in metric['results']:
        for timeseries in result['timeseries']:
            new_dict = {}
            new_dict['aggregator'] = metric['aggregator']
            new_dict['event_id'] = metric['event_id']
            new_dict['name'] = metric['name']
            new_dict['scope'] = metric['scope']
            new_dict['winning_direction'] = metric['winning_direction']

            new_dict['result_is_baseline'] = result['is_baseline']
            new_dict['result_level'] = result['level']
            new_dict['result_name'] = result['name']
            new_dict['result_variation_id'] = result['variation_id']
            new_dict['result_id'] = result['results_id']
            
            if 'rate' in timeseries.keys():
                new_dict['time_series_rate'] = timeseries['rate']
            else:
                pass
            
            new_dict['time_series_time'] = timeseries['time']
            new_dict['time_series_value'] = timeseries['value']
            new_dict['time_series_samples'] = timeseries['samples']
            new_dict['time_series_variance'] = timeseries['variance']

            flattened_metrics.append(new_dict)
new_j_ts['metrics'] = flattened_metrics
new_j_ts['upload_ts'] = str(datetime.now())

In [69]:
new_j_ts

{'confidence_threshold': 0.9,
 'end_time': '2018-06-05T16:35:00.000000Z',
 'experiment_id': 10796678057,
 'metrics': [{'aggregator': 'unique',
   'event_id': 10757331770,
   'name': 'Create new account button',
   'result_id': '10809912117',
   'result_is_baseline': True,
   'result_level': 'variation',
   'result_name': 'Original',
   'result_variation_id': '10809912117',
   'scope': 'visitor',
   'time_series_samples': 0,
   'time_series_time': '2018-06-05T00:21:32.000000Z',
   'time_series_value': 0,
   'time_series_variance': 0.0,
   'winning_direction': 'increasing'},
  {'aggregator': 'unique',
   'event_id': 10757331770,
   'name': 'Create new account button',
   'result_id': '10809912117',
   'result_is_baseline': True,
   'result_level': 'variation',
   'result_name': 'Original',
   'result_variation_id': '10809912117',
   'scope': 'visitor',
   'time_series_rate': 0.0,
   'time_series_samples': 1,
   'time_series_time': '2018-06-05T00:31:16.080000Z',
   'time_series_value': 0,